In [2]:
import gym
from gym import Env
from gym.spaces import Box, Dict, Tuple, MultiBinary, MultiDiscrete, Discrete
import numpy as np
import random
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv 
from stable_baselines3.common.evaluation import evaluate_policy

In [15]:
class ShowerEnv(Env):
    def __init__(self):
        self.action_space = Discrete(3)
        self.observation_space = Box(low=0, high=100, shape=(1,))
        self.state = 38 + random.randint(-3, 3)
        self.shower_length = 60

    def step(self, action):
        self.state += action-1
        self.shower_length -= 1

        if self.state >=37 and self.state <= 39:
            reward = 1
        else:
            reward = -1
        
        if self.shower_length <=0:
            done = True
        else:
            done = False
        
        info = {}
        return self.state, reward, done, info
    def render(self):
        pass
    def reset(self):
        self.state = np.array([38+random.randint(-3,3)]).astype(float)
        self.shower_length = 60
        return self.state

In [16]:
env = ShowerEnv()

In [140]:
env.action_space.sample()

2

In [141]:
episode = 5
for n in range(1,episode+1):
    obs = env.reset()
    score = 0
    done = False
    while not done:
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        score += reward
    print(f"episode - {n} || score - {score}")

episode - 1 || score - -58
episode - 2 || score - -60
episode - 3 || score - -40
episode - 4 || score - -56
episode - 5 || score - 18
